In [61]:
from google.colab import drive
drive.mount('/content/drive')
import os
!pip install opencv-python
%matplotlib inline
import typing as tp
from matplotlib import pyplot as plt
from pathlib import Path
import numpy as np
from google.colab.patches import cv2_imshow

# Change the current working directory
os.chdir('/content/drive/My Drive/CV_')

# Verify the current working directory
print("Current directory:", os.getcwd())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current directory: /content/drive/My Drive/CV_


In [67]:
import cv2
import numpy as np

# Read the input images and the mask
mask = cv2.imread("/content/drive/My Drive/CV_/figure4-assignment3.jpg", cv2.IMREAD_GRAYSCALE)
image1 = cv2.imread("/content/drive/My Drive/CV_/figure2-assignment3.jpg")
image2 = cv2.imread("/content/drive/My Drive/CV_/figure3-assignment3.jpg")

# Set the level of the pyramids
levels=6

# Resize images to match mask dimensions
height, width = image1.shape[:2]
mask = cv2.resize(mask, (width, height), interpolation=cv2.INTER_LINEAR)

# Duplicate the mask to match the number of channels in the images
mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

# Generate Gaussian pyramids for both images and the mask
gaussian_pyramid1 = [image1.astype(np.float32)]
gaussian_pyramid2 = [image2.astype(np.float32)]
mask_pyramid = [mask.astype(np.float32) / 255.0]

for _ in range(levels - 1):
    image1 = cv2.pyrDown(image1)
    image2 = cv2.pyrDown(image2)
    mask = cv2.pyrDown(mask)

    gaussian_pyramid1.append(image1.astype(np.float32))
    gaussian_pyramid2.append(image2.astype(np.float32))
    mask_pyramid.append(mask.astype(np.float32) / 255.0)

# Generate Laplacian pyramids for both images
laplacian_pyramid1 = [gaussian_pyramid1[levels - 1]]
laplacian_pyramid2 = [gaussian_pyramid2[levels - 1]]
for i in range(levels - 2, -1, -1):
    expanded1 = cv2.pyrUp(gaussian_pyramid1[i + 1], dstsize=(gaussian_pyramid1[i].shape[1], gaussian_pyramid1[i].shape[0]))
    expanded2 = cv2.pyrUp(gaussian_pyramid2[i + 1], dstsize=(gaussian_pyramid2[i].shape[1], gaussian_pyramid2[i].shape[0]))
    laplacian1 = cv2.subtract(gaussian_pyramid1[i], expanded1)
    laplacian2 = cv2.subtract(gaussian_pyramid2[i], expanded2)
    laplacian_pyramid1.append(laplacian1)
    laplacian_pyramid2.append(laplacian2)

# Combine the corresponding levels of Laplacian pyramids using the mask
composite_pyramid = []
for laplacian1, laplacian2, mask in zip(laplacian_pyramid1, laplacian_pyramid2, mask_pyramid):
    mask_resized = cv2.resize(mask, (laplacian1.shape[1], laplacian1.shape[0]), interpolation=cv2.INTER_LINEAR)
    composite_level = laplacian1 * (1.0 - mask_resized) + laplacian2 * mask_resized
    composite_pyramid.append(composite_level)

# Reconstruct the final blended image
composite_image = composite_pyramid[0]
for i in range(1, levels):
    composite_image = cv2.pyrUp(composite_image, dstsize=(composite_pyramid[i].shape[1], composite_pyramid[i].shape[0]))
    composite_image += composite_pyramid[i]

# Ensure pixel values are within valid range
composite_image = np.clip(composite_image, 0, 255).astype(np.uint8)

# Save Image
cv2.imwrite("/content/drive/My Drive/CV_/composite_image.jpg", composite_image)

# Brighntness factor to fine tune the output image
brightness_factor = -9  # Adjust this value as needed
brighter_image = cv2.convertScaleAbs(composite_image, alpha=1, beta=brightness_factor)
brighter_image = np.clip(brighter_image, 0, 255)

# Save the brighter image
cv2.imwrite("/content/drive/My Drive/CV_/Final_output_image.jpg", brighter_image)


True

True

In [ ]:
\